In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from itertools import compress

import pickle

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from collections import Counter

import matplotlib.pyplot as plt
%matplotlib inline

import plotly.express as px
from sklearn.decomposition import PCA

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/laurenshores/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# This data was compiled by copy/pasting a table from wikipedia
# https://en.wikipedia.org/wiki/List_of_The_Amazing_Race_(American_TV_series)_contestants#

data = pd.read_csv("~/Documents/Data_Science_Projects/Amazing_Race/Amazing_Race_Contestants.csv")


In [3]:
# get contestant teams in one row using a self join

dat = data.merge(data.drop(columns='Special_Season'), on=["Season", "Race_Place"]).drop_duplicates(['Name_y', 'Season', 'Race_Place'])
dat["drop_ind"] = dat['First_Name_x'] == dat['First_Name_y']
dat = dat[dat["drop_ind"] == False].drop(columns='drop_ind').reset_index(drop=True)
print(len(dat))
dat.head(3)

414


,Name_x,First_Name_x,Age_x,Hometown_x,Season,Race_Place,Special_Season,Name_y,First_Name_y,Age_y,Hometown_y
0,Matt Robar,Matt,28,"Simsbury, Connecticut",Season 1,11th,NaN,Ana Robar,Ana,28,"Simsbury, Connecticut"
1,Kim Smith,Kim,28,"Baytown, Texas",Season 1,10th,NaN,Leslie Kellner,Leslie,27,"Baytown, Texas"
2,Pat Pierce,Pat,43,"Landenberg, Pennsylvania",Season 1,9th,NaN,Brenda Mehta,Brenda,42,"Elkton, Maryland"


# Scraping More Info

This section scrapes data from a fandom site and saves scraped df to a csv.

* Create a list of possible links to scrape using format: https://amazingrace.fandom.com/wiki/Matt_%26_Ashley
* Create the scraper function

In [4]:
# Create links list

x_names = list(dat['First_Name_x'])
y_names = list(dat['First_Name_y'])
links = []
for i in range(len(dat)):

    path = 'https://amazingrace.fandom.com/wiki/' + x_names[i] + '_%26_' + y_names[i]
    links.append(path)

In [5]:
# checking out beautiful soup
response=requests.get(links[0])

response.text

soup=BeautifulSoup(response.content, 'html.parser')

soup

<!DOCTYPE html>

<html class="client-nojs sse-other" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Matt &amp; Ana | The Amazing Race Wiki | Fandom</title>
<script>document.documentElement.className="client-js sse-other";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"1f74cb00568b797fea2fe2222121a47d","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Matt_\u0026_Ana","wgTitle":"Matt \u0026 Ana","wgCurRevisionId":154226,"wgRevisionId":154226,"wgArticleId":2171,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Facebook","Teams","The Amazing Race (US)","The Amazing Race (US) teams","The Amazing Race (US) 1","The Amazing Race (US) 

In [6]:
# Get the team page links: https://amazingrace.fandom.com/wiki/Category:The_Amazing_Race_(US)_teams

us_teams_urls = ['https://amazingrace.fandom.com/wiki/Category:The_Amazing_Race_(US)_teams',
                'https://amazingrace.fandom.com/wiki/Category:The_Amazing_Race_(US)_teams?from=Laurence+%26+Zac']

team_links = []
for url in us_teams_urls:
    res=requests.get(url)

    soup1=BeautifulSoup(res.content, 'html.parser')
    
    teams = soup1.find_all("div", {"class":"category-page__member-left"})
    for t in teams:
        link_ending = t.find_all("a")#, {"href":'wiki(.+?)'})#.div.a
        lnk = re.search('wiki(.+?)"',str(link_ending) ).group(1)
        team_links.append(lnk)

lnks = ['https://amazingrace.fandom.com/wiki' + i for i in team_links]
print("Number of Team Links: ", len(lnks))

AttributeError: 'NoneType' object has no attribute 'group'

In [ ]:
#if 'Placement' in d:
 #   print(True)
#print(d)

tst_lnks = ['https://amazingrace.fandom.com/wiki/Aaron_%26_Arianne', 'https://amazingrace.fandom.com/wiki/Amanda_%26_Kris', 
            'https://amazingrace.fandom.com/wiki/Oswald_%26_Danny', 'https://amazingrace.fandom.com/wiki/Amani_%26_Marcus'
           ]

In [ ]:
# Putting it together

dic_list = []
profile_list = []
profile_list2 = []

for t in range(len(lnks)):
    
    response=requests.get(lnks[t])
    print(lnks[t])
    
    soup=BeautifulSoup(response.content, 'html.parser')
    

    # Getting relationship/occupation,etc
    # Initialize a dictionary,
    # loop through the range of returned tags
    # use re to search the string pattern that returns either the label or the value (position them in dic accordingly)
    # pull group 1 from returned search to get just the value after the found string

    # We will create a list of dictionaries, 1 for each team/page we scrape

    d={}
    d['team'] = lnks[t].split('/')[-1]
    
    # using site categories to add some data points
    soup_str = str(soup)
    
    try:
        cats = re.search('"wgCategories":(.+?)]',soup_str).group(1)[1:]
    
        cats =cats.replace('"', '').split(",")

        # get the season #
        season_search =[ re.search('The Amazing Race (.+?)teams', i) for i in cats]
        season_list=list(compress(cats, season_search))
        #d['Season'] = [int(i) for i in season_list[1].split() if i.isdigit()][0]
        # there is and "Amazing Race Teams category that does not have season info first in the list. Get rid of it"
        season_list = season_list[1:]

        seasons = []
        for l in season_list:
            for t in l.split():
                if t.isdigit():
                    seasons.append(t)

        d['Season'] = seasons


        # searching for and getting rid of some categories that could blow out the width of the dataframe
        cats_bool =[not re.search('The Amazing Race (.+?)', i) for i in cats]
        cats=list(compress(cats, cats_bool))
        cats_bool2 =[not re.search('(.+?) place', i) for i in cats]
        cats=list(compress(cats, cats_bool2))

        for i in cats:
            d[i] = 1
    except:
        None #print('no categories')
    
    # Going for season info using a different search pattern to catch edge cases
    try:
        cats = re.search('"wgCategories":(.+)',soup_str).group(1)[1:]

        cats =cats.replace('"', '').split(",")

        # get the season #
        season_search =[ re.search('The Amazing Race (.+?)teams', i) for i in cats]
        season_list=list(compress(cats, season_search))
        #d['Season'] = [int(i) for i in season_list[1].split() if i.isdigit()][0]
        # there is and "Amazing Race Teams category that does not have season info first in the list. Get rid of it"
        season_list = season_list[1:]

        seasons = []
        for l in season_list:
            for t in l.split():
                if t.isdigit():
                    seasons.append(t)

        d['Season'] = seasons


        # searching for and getting rid of some categories that could blow out the width of the dataframe
        cats_bool =[not re.search('The Amazing Race (.+?)', i) for i in cats]
        cats=list(compress(cats, cats_bool))
        cats_bool2 =[not re.search('(.+?) place', i) for i in cats]
        cats=list(compress(cats, cats_bool2))

        for i in cats:
            d[i] = 1
    except:
        None

    
    #--- data points in step with site labels
    for i in range(len(soup.find_all("h3", {"class": "pi-data-label pi-secondary-font"}))):
        #print(str(i))
        key = re.search(' pi-secondary-font">(.+?):</h3>',\
                   str(soup.find_all("h3", {"class": "pi-data-label pi-secondary-font"})[i])).group(1)
        value = re.search(' pi-font">(.+?)</div>', \
                   str(soup.find_all("div",{"class": "pi-data-value pi-font"})[i])).group(1)
        if key in d:
            d[key] = [d[key]] + [value]
        else:
            d[key] = value
         
    try:
        del d['Born']
    except:
        None #print("no born info")
    try:
        del d['Countries<br/>Raced']
    except:
        None #print("no countries info")
    
    cats_to_delete = ['Instagram','Twitter','Teams','Facebook','Official Website', 'Penalized teams',
                      'Fast Forward users', 'Non-Elimination Leg Survivors', 'wgPageContentLanguage:en',
                     'wgPageContentModel:', 'LinkedIn', 'Wikipedia']
    for c in cats_to_delete:
        try:
            del d[c]
        except:
            None #print("no categories info for: ", c)
    dic_list.append(d)

    #---------------------------

    # Getting profile text
    # Get the response text, find the indexes of the following tag strings using re, slice the response using the index
    # reconvert the slice to soup object and use 'p' to find the text.

    # Each team will have a list of text. The first two items in text should be each player's profile responses

    txt = response.text
    

    # nested/double search to get exactly the index number
    try:
        start_txt = re.search('span=(.+?),', str(re.search('<h3><span class="mw-headline"', txt) ) ).group(1)[1:]
        
        try:
            end_txt = re.search('span=(.+?),', str(re.search('<sup id="cite_ref-2"',txt) ) ).group(1)[1:]
        except:
            None
        
        if end_txt < start_txt:
            # if the tag above isn't found, just grab the next 5k characters
            end_txt = int(start_txt) + 5000
        else:
            end_txt = end_txt
        
        txt1 = txt[int(start_txt):int(end_txt) ]

        text_list = []
        txt1 = BeautifulSoup(txt1, "html.parser")
        for para in txt1.find_all("p"):
            text_list.append(para.get_text())
    
    
    except:
        text_list = ["no profile info"]
        #print("no profile")

    
    profile_list.append(text_list)
    
    # some profiles don't fall under this pattern...make another list for a profile 2 -------------------
    
    # nested/double search to get exactly the index number
    
    try:
        start_txt = re.search('span=(.+?),', str(re.search('<div class="tabber wds-tabber"', txt) ) ).group(1)[1:]
        
        try:
            end_txt = re.search('span=(.+?),', str(re.search('<h3><span class="mw-headline"',txt) ) ).group(1)[1:]
        except:
            None
        
        if end_txt < start_txt:
            # if the tag above isn't found, just grab the next 5k characters
            end_txt = int(start_txt) + 5000
        else:
            end_txt = end_txt
        
        txt2 = txt[int(start_txt):int(end_txt) ]

        text_list2 = []
        txt2 = BeautifulSoup(txt2, "html.parser")
        for para in txt2.find_all("p"):
            text_list2.append(para.get_text())
        for para in txt2.find_all("li"):
            text_list2.append(para.get_text())
    
    
    except:
        text_list2 = ["no profile info"]
        

    
    profile_list2.append(text_list2)
    


In [ ]:
print("number of links: ", len(lnks))
print("dictionary length: ", len(dic_list))
df =pd.DataFrame(dic_list)

df['profile_txt'] = profile_list
df['profile_txt2'] = profile_list2


# Open a file and use dump() 
#with open('scraped_df.pkl', 'wb') as file: 
      
    # A new file will be created 
 #   pickle.dump(df, file)




In [ ]:
# Need to be able to take rows that have > 1 seasons and placements and break into multiple columns
print("Teams without Season info: ", len(df[df["Season"].isna()]))
print("Teams without Placement info: ", len(df[df["Placement"].isna()]))


In [ ]:
#df.to_csv('scraped_df.csv', index=False)


# Join Wikipedia and Fandom Data

Can probably do a solid join on Season and Place. data from df will be duplicated to fit to dat.
Once get in one frame, can go through and clean up the text in columns that want to put to use.
["Relation", "Occupation","profile_txt"]

Then go through the dataframe and look for columns that mean same thing and combine. Ex: LGBT Teams, Lesbian teams, Gay Teams, Gay/Bisexual Teams

In [4]:
df = pd.read_csv("scraped_df.csv")

# first thing: get rid of columns that start with 'wgRelevantPageName'
df.drop(list(df.filter(regex='wgRelevant|wgPage|wgIs')), axis=1, inplace=True)

In [27]:
len(df)

350

In [5]:
df.head()

,team,Season,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,...,Linkedin,Straight/Bisexual Teams],Hispanic-American Teams,Gay Teams],Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2
0,Aaron_%26_Arianne,['3'],1.0,1.0,1.0,1.0,"New York City, New York",Lifelong friends,Pre-school teacher (Aaron)<br/>Waitress (Arianne),6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],"['\n', 'Aaron and Arianne have been friends fo..."
1,Aastha_%26_Nina,['34'],NaN,1.0,NaN,NaN,"Marina Del Ray, California",Engaged,VP of Operations (Aastha)<br/>Director of Busi...,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],['no profile info']
2,Abbie_%26_Ryan,['21'],1.0,NaN,NaN,NaN,"Encinitas, California (Abbie)<br/>San Diego, C...",Dating Divorcees,Dance Instructor/Choreographer (Abbie)<br/>Fin...,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Age: 31\nHometown: Encinitas, Calif.\nConnec...",['no profile info']
3,Abby_%26_Will,['34'],1.0,NaN,NaN,NaN,"Birmingham, Alabama",Dating,Data Scientist (Abby)<br/>Accountant (Will),5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""Notes:\n\n1: Leg 3 was a megaleg, with two D...",['no profile info']
4,Adam_%26_Bethany,['25'],1.0,NaN,NaN,NaN,"Princeville, Hawaii",Married Surfers,Professional Husband/ Full-time support (Adam)...,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Age: 26\nHometown: Princeville, Hawaii\nConn...",['no profile info']


## Looking out for rows/teams that did multiple seasons

In [6]:
df['Placement']

0                 7th
1                12th
2                 5th
3                 8th
4                 3rd
            ...      
345               9th
346               3rd
347               1st
348               6th
349    ['9th', '4th']
Name: Placement, Length: 350, dtype: object

In [7]:


def season_to_list(x):
    
    # Remove the brackets and split the string by comma
    x_list = x[1:-1].split(',')

    # Strip whitespace and convert elements to integers if needed
    x_list = [int(element.replace("'","").strip()) for element in x_list]
    
    return x_list

In [8]:
df['Season'] = df['Season'].apply(lambda x: season_to_list(x))

In [9]:
multi_racers = df[df['Season'].apply(lambda x: len(x)) > 1 ]
multi_racers.head(3)



,team,Season,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,...,Linkedin,Straight/Bisexual Teams],Hispanic-American Teams,Gay Teams],Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2
16,Amanda_%26_Kris,"[14, 18]",1.0,NaN,NaN,NaN,"San Diego, California (Season 14)<br/>Prismo B...",Dating (Season 14)<br/>Engaged (Unfinished Bus...,Student; College Admissions at a Private Colle...,"['4', '2']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Age: 25\nHometown:\xa0Redwood City, Calif. b...","['\n', 'Amanda Blackledge (23); Kris Klicka (2..."
30,Art_%26_JJ,"[20, 31]",NaN,NaN,NaN,1.0,"Temecula, California (Art)<br/>Carlsbad, Calif...",Best Friends,Border Patrol Agents,"['12', '1']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Age:\xa043\nHometown:\xa0Temecula, Calif.\nC...","['\n', 'The Amazing Race 20', 'The Amazing Rac..."
36,Becca_%26_Floyd,"[29, 31]",1.0,NaN,NaN,1.0,"Pittsburgh, Pennsylvania (Becca [TAR29])<br/>B...",Strangers (TAR29)<br/>Friends (TAR31),"Rock Climbing Instructor, Coach, Guide, and Ba...","['10', '10']",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""Age: 26\nHometown: Pittsburgh, Penn.\nCurren...","['\n', 'The Amazing Race 29', 'The Amazing Rac..."


In [10]:
df1 = df[df['Season'].apply(lambda x: len(x)) == 1 ]

df1['Season_1st'] =df1['Season'].apply(lambda x: x[0])
df1['Season_2nd'] = 0
df1['Placement_1st'] = df1['Placement']
df1['Placement_2nd'] = 0

df1.head(3)

/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3324355037.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Season_1st'] =df1['Season'].apply(lambda x: x[0])
/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3324355037.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Season_2nd'] = 0
/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3324355037.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

,team,Season,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,...,Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2,Season_1st,Season_2nd,Placement_1st,Placement_2nd
0,Aaron_%26_Arianne,[3],1.0,1.0,1.0,1.0,"New York City, New York",Lifelong friends,Pre-school teacher (Aaron)<br/>Waitress (Arianne),6,...,NaN,NaN,NaN,NaN,[],"['\n', 'Aaron and Arianne have been friends fo...",3,0,7th,0
1,Aastha_%26_Nina,[34],NaN,1.0,NaN,NaN,"Marina Del Ray, California",Engaged,VP of Operations (Aastha)<br/>Director of Busi...,1,...,NaN,NaN,NaN,NaN,[],['no profile info'],34,0,12th,0
2,Abbie_%26_Ryan,[21],1.0,NaN,NaN,NaN,"Encinitas, California (Abbie)<br/>San Diego, C...",Dating Divorcees,Dance Instructor/Choreographer (Abbie)<br/>Fin...,9,...,NaN,NaN,NaN,NaN,"['Age: 31\nHometown: Encinitas, Calif.\nConnec...",['no profile info'],21,0,5th,0


Method:
Can make a column for season 1 and season 2, placement 1 and placement 2. There doesn't seem to be >2 racers

* could take all the multi_racers out and join the single racers df with dat
* Then for multi_racers could do 2 joins. 1 on season/place 1. Then on season/place2. That way all season/places from dat should have a match from df/multi_racers

In [11]:
multi_racers['Season_1st'] = multi_racers['Season'].apply(lambda x: x[0])
multi_racers['Season_2nd'] = multi_racers['Season'].apply(lambda x: x[1])
multi_racers['Placement_1st'] = multi_racers['Placement'].apply(lambda x: x[0])
multi_racers['Placement_2nd'] = multi_racers['Placement'].apply(lambda x: x[1])

/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3524035879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_racers['Season_1st'] = multi_racers['Season'].apply(lambda x: x[0])
/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3524035879.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  multi_racers['Season_2nd'] = multi_racers['Season'].apply(lambda x: x[1])
/var/folders/hd/cqjrh15x0097m3t8zb9vmvnr0000gn/T/ipykernel_90225/3524035879.py:3: SettingWithCopyWarning: 
A

In [12]:
multi_racers.head(3)

,team,Season,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,...,Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2,Season_1st,Season_2nd,Placement_1st,Placement_2nd
16,Amanda_%26_Kris,"[14, 18]",1.0,NaN,NaN,NaN,"San Diego, California (Season 14)<br/>Prismo B...",Dating (Season 14)<br/>Engaged (Unfinished Bus...,Student; College Admissions at a Private Colle...,"['4', '2']",...,NaN,NaN,NaN,NaN,"['Age: 25\nHometown:\xa0Redwood City, Calif. b...","['\n', 'Amanda Blackledge (23); Kris Klicka (2...",14,18,[,'
30,Art_%26_JJ,"[20, 31]",NaN,NaN,NaN,1.0,"Temecula, California (Art)<br/>Carlsbad, Calif...",Best Friends,Border Patrol Agents,"['12', '1']",...,NaN,NaN,NaN,NaN,"['Age:\xa043\nHometown:\xa0Temecula, Calif.\nC...","['\n', 'The Amazing Race 20', 'The Amazing Rac...",20,31,[,'
36,Becca_%26_Floyd,"[29, 31]",1.0,NaN,NaN,1.0,"Pittsburgh, Pennsylvania (Becca [TAR29])<br/>B...",Strangers (TAR29)<br/>Friends (TAR31),"Rock Climbing Instructor, Coach, Guide, and Ba...","['10', '10']",...,NaN,NaN,NaN,NaN,"[""Age: 26\nHometown: Pittsburgh, Penn.\nCurren...","['\n', 'The Amazing Race 29', 'The Amazing Rac...",29,31,[,'


In [13]:
print(multi_racers.iloc[:,-4:].dtypes)
print("---------------------")
print(df1.iloc[:,-4:].dtypes)

Season_1st        int64
Season_2nd        int64
Placement_1st    object
Placement_2nd    object
dtype: object
---------------------
Season_1st        int64
Season_2nd        int64
Placement_1st    object
Placement_2nd     int64
dtype: object


* bring the datasets back together.
* Season1st and 2nd to integers.
* Placement 1st and 2nd columns need to be processed to remove 'th','rd','st' before can turn to int

Then can do the tow different df joins (inner) with dat

In [14]:
# bring the datasets back together.
# Season1st and 2nd to integers.
# Placement 1st and 2nd columns need to be processed to remove 'th','rd','st' before can turn to int
df_full = pd.concat([df1, multi_racers])
df_full['Placement_1st'] = df_full['Placement_1st'].astype(str)
df_full['Placement_2nd'] = df_full['Placement_2nd'].astype(str)

In [15]:
df_full[["Season",'Placement_1st','Placement_2nd']]

,Season,Placement_1st,Placement_2nd
0,[3],7th,0
1,[34],12th,0
2,[21],5th,0
3,[34],8th,0
4,[25],3rd,0
...,...,...,...
297,"[12, 18]",[,'
325,"[3, 11]",[,'
340,"[28, 31]",[,'
341,"[7, 11]",[,'


In [16]:
def myfun(mystring, top):
    """ this function will loop through a reversed range of numbers and pull out the first matching number
    in the range that occurs inside of a mystring. Originally used to format place column, 
    but also helpful for season"""
    place = reversed(range(1,top))
    for i in place:
        if str(i) in mystring:
            return i

dat["Season1"] =dat["Season"].apply(lambda x: myfun(x,36))
df_full["Season_1st"] = df_full["Season_1st"].astype(int)
df_full["Season_2nd"] = df_full["Season_2nd"].astype(int)
#---
df_full["Placement_1st"] = df_full.Placement_1st.apply(lambda x: myfun(x,15))
df_full["Placement_2nd"] = df_full.Placement_2nd.apply(lambda x: myfun(x,15))
dat["RPlace"] = dat["Race_Place"].apply(lambda x: myfun(x,15))

df_full['Placement_2nd'].fillna(0, inplace=True)

In [17]:
df_full.tail(3)

,team,Season,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,...,Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2,Season_1st,Season_2nd,Placement_1st,Placement_2nd
340,Tyler_%26_Korey,"[28, 31]",NaN,1.0,NaN,1.0,"Jackson, Michigan;<br/>Los Angeles, California...",Best Friends,Online Video Personality/Author/Host (Tyler)<b...,"['12', '12']",...,NaN,NaN,NaN,NaN,['Name: Tyler Oakley\nAge: 26\nTwitter: @tyler...,"['\n', 'The Amazing Race 28', 'The Amazing Rac...",28,31,NaN,0
341,Uchenna_%26_Joyce,"[7, 11]",1.0,NaN,NaN,NaN,"Houston, Texas",Married,Energy broker (Uchenna)<br/>Sales manager (Joyce),"['12', '9']",...,NaN,NaN,NaN,NaN,[],"['\n', ""Uchenna & Joyce are a married Team fro...",7,11,NaN,0
349,Zev_%26_Justin,"[15, 18]",NaN,NaN,NaN,1.0,"Sherman Oaks, California (Zev)<br/>Los Angeles...",Friends (Season 15)<br/>Best Friends (Unfinish...,Actor (Zev)<br/>VP of Production and Developme...,"['4', '11']",...,NaN,NaN,NaN,NaN,"[""Age: 28\nHometown:\xa0Sherman Oaks, Calif.\n...","['\n', 'Zev Glassenberg (26); Justin Kanew (30...",15,18,NaN,0


In [18]:
pd.options.display.max_columns=300
dat_a = dat.merge(df_full, how='inner', left_on=['Season1','RPlace'], right_on=['Season_1st','Placement_1st'])
print(len(dat_a))
# this returns more teams than expected, but seems to be due to their being >2 people in team/families.
#dat_a[dat_a.duplicated(['Name_x','First_Name_x','Age_x','Hometown_x','Season_x','Race_Place'])]


dat_b = dat.merge(df_full, how='inner', left_on=['Season1','RPlace'], right_on=['Season_2nd','Placement_2nd'])
#dat_b[dat_b.duplicated(['Name_x','First_Name_x','Age_x','Hometown_x','Season_x','Race_Place'])]
# dat_b does not seem to raise any issues as far as duplicates
print(len(dat_b))

339
0


In [19]:
#remove the family teams and other odd duplicated teams.
# these would only serve to muddle up results since they aren't two-person teams and often with kids
dups_a =dat_a[dat_a.duplicated(['Name_x','First_Name_x','Age_x','Hometown_x','Season_x','Race_Place'])]

print("length of dat_a: ", len(dat_a))
print("length of duplicates: ", len(dups_a))

dat_a = dat_a.drop(dups_a.index)
print("length of new data frame: ",len(dat_a) )

length of dat_a:  339
length of duplicates:  26
length of new data frame:  313


In [20]:
dat1=pd.concat([dat_a, dat_b], ignore_index=True)

In [21]:
len(dat1)

313

In [22]:
dat1.head()

,Name_x,First_Name_x,Age_x,Hometown_x,Season_x,Race_Place,Special_Season,Name_y,First_Name_y,Age_y,Hometown_y,Season1,RPlace,team,Season_y,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,Legs Won,Placement,Average Position,Roadblock<br/>Count,All-female teams,Asian-American Teams,Engaged teams,Lesbian teams,Interracial Teams,Interracial Teams],Unnamed: 20,Version,Dating teams,U-Turned teams,Medevac,Disabled teams,Married teams,Express Pass users,Finalists,Hashtag,Internet Movie Database,Exes teams,Yield users,Yielded teams,All-male teams,Family teams,In-law teams,African-American Teams,African-American teams,Cousins teams,Disabled teams],U-Turn users,Sibling teams,Reality Show Teams,Work-related teams,Speed Bump performers,Reality Show Teams],Winners,Winners],Parent/Child teams,YouTube,Jewish Teams,Jewish Teams],Discharged teams,Discharged teams],Penalized teams],Stranger teams,Gay Teams,Blind Dating teams,Stranger teams],Muslim Teams,Muslim Teams],Deceased,Deceased],Died,LGBT Winners,LGBT Winners],Dating teams],Gay/Bisexual Teams,Lesbian teams],LGBT teams],Quitters,Quitters],Straight/Bisexual Teams,Twin sibling teams,Composite teams,Returning racers,Date Night users,Hazard performers,Social Media,Non-Elimination Leg Survivors],Linkedin,Straight/Bisexual Teams],Hispanic-American Teams,Gay Teams],Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2,Season_1st,Season_2nd,Placement_1st,Placement_2nd
0,Matt Robar,Matt,28,"Simsbury, Connecticut",Season 1,11th,NaN,Ana Robar,Ana,28,"Simsbury, Connecticut",1,11,Matt_%26_Ana,[1],NaN,NaN,NaN,NaN,"Simsbury, Connecticut",Married,Military folk,1,0,11th,11.00,Matt: 1<br/>Ana: 0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""I'd most like to receive a letter from: Nobo...",['no profile info'],1,0,11.0,0
1,Kim Smith,Kim,28,"Baytown, Texas",Season 1,10th,NaN,Leslie Kellner,Leslie,27,"Baytown, Texas",1,10,Kim_%26_Leslie,[1],NaN,NaN,NaN,1.0,"Baytown, Texas",Roommates,Teachers,2,0,10th,8.00,Kim: 2<br/>Leslie: 0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""I'd most like to receive a letter from: Caro...",['no profile info'],1,0,10.0,0
2,Pat Pierce,Pat,43,"Landenberg, Pennsylvania",Season 1,9th,NaN,Brenda Mehta,Brenda,42,"Elkton, Maryland",1,9,Pat_%26_Brenda,[1],NaN,NaN,NaN,1.0,"Landenberg, Pennsylvania (Pat)<br/>Elkton, Mar...",Working moms,NaN,3,1,9th,5.00,Pat: 2<br/>Brenda: 0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""I'd most like to receive a letter from: My k...",['no profile info'],1,0,9.0,0
3,Dave Groark,Dave,65,"Rockwall, Texas",Season 1,8th,NaN,Margaretta Groark,Margaretta,60,"Rockwall, Texas",1,8,Dave_%26_Margaretta,[1],1.0,NaN,NaN,NaN,"Rockwall, Texas",Married,Air Force Pilot (Dave)<p>Retired (Margaretta)</p>,4,0,8th,6.50,Dave: 3<br/>Margaretta: 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,"October 26, 2008 (age 67) (Margaretta)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[""I'd most like to receive a letter from: My k...",['no pro

### Checks for the merge/missing data

This is still ~40 teams less than the df that was pulled from wikipedia, which seems to be just what was available on the fandom site. May want to do some spot checking and see which teams still didn't pick up and if they have a fandom site page.

In [23]:
# to see how many teams from wikipedia didn't match with fandom
q = dat.merge(dat1, how='left', on=['Season1','RPlace'])
print(len(q[q['Season_1st'].isna()]))
q[q['Season_1st'].isna()]

75


,Name_x_x,First_Name_x_x,Age_x_x,Hometown_x_x,Season,Race_Place_x,Special_Season_x,Name_y_x,First_Name_y_x,Age_y_x,Hometown_y_x,Season1,RPlace,Name_x_y,First_Name_x_y,Age_x_y,Hometown_x_y,Season_x,Race_Place_y,Special_Season_y,Name_y_y,First_Name_y_y,Age_y_y,Hometown_y_y,team,Season_y,Co-ed teams,LGBT teams,Straight/Gay Teams,Friend teams,Hometown,Relation,Occupation,Legs Competed,Legs Won,Placement,Average Position,Roadblock<br/>Count,All-female teams,Asian-American Teams,Engaged teams,Lesbian teams,Interracial Teams,Interracial Teams],Unnamed: 20,Version,Dating teams,U-Turned teams,Medevac,Disabled teams,Married teams,Express Pass users,Finalists,Hashtag,Internet Movie Database,Exes teams,Yield users,Yielded teams,All-male teams,Family teams,In-law teams,African-American Teams,African-American teams,Cousins teams,Disabled teams],U-Turn users,Sibling teams,Reality Show Teams,Work-related teams,Speed Bump performers,Reality Show Teams],Winners,Winners],Parent/Child teams,YouTube,Jewish Teams,Jewish Teams],Discharged teams,Discharged teams],Penalized teams],Stranger teams,Gay Teams,Blind Dating teams,Stranger teams],Muslim Teams,Muslim Teams],Deceased,Deceased],Died,LGBT Winners,LGBT Winners],Dating teams],Gay/Bisexual Teams,Lesbian teams],LGBT teams],Quitters,Quitters],Straight/Bisexual Teams,Twin sibling teams,Composite teams,Returning racers,Date Night users,Hazard performers,Social Media,Non-Elimination Leg Survivors],Linkedin,Straight/Bisexual Teams],Hispanic-American Teams,Gay Teams],Speed Bump performers],Work-related teams],Friend teams],Greeters,profile_txt,profile_txt2,Season_1st,Season_2nd,Placement_1st,Placement_2nd
7,Kevin O'Connor,Kevin,34,"Bayonne, New Jersey",Season 1,4th,NaN,Drew Feinberg,Drew,35,"Staten Island, New York",1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Joe Baldassare,Joe,50,"Laguna Niguel, California",Season 1,3rd,NaN,Bill Bartek,Bill,47,"Laguna Niguel, California",1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,Oswald Mendez,Oswald,31,"Miami, Florida",Season 2,4th,NaN,Danny Jimenez,Danny,36,"Miami, Florida",2,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,John Vito Pietanza,John,28,"Staten Island, New York",Season 3,5th,NaN,Jill Aquilino,Jill,24,"Manhattan, New York",3,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,Teri Pollack,Teri,49,"Palm City, Florida",Season 3,2nd,NaN,Ian Pollack,Ian,50,"Palm City, Florida",3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [24]:
# check out what's missing
# count missing values per column. MAKE A NEW LIST OF RELEVANT COLUMNS

z=dat1[["First_Name_x", "First_Name_y","Age_x", "Hometown_x","Age_y", "Hometown_y","RPlace","Season1", "Relation", "Occupation","profile_txt", "profile_txt2"]]

missing_col = [z[i].isna().sum() for i in z.columns]
#census_df.isna().sum( axis=0)

missing_perc = [i/len(z) for i in missing_col]
missing_perc_df = pd.DataFrame({"col_name": z.columns, "prop_missing": missing_perc}).sort_values(by=['prop_missing'], ascending=False)

missing_perc_df.head(75)

,col_name,prop_missing
9,Occupation,0.006390
8,Relation,0.003195
0,First_Name_x,0.000000
1,First_Name_y,0.000000
2,Age_x,0.000000
3,Hometown_x,0.000000
4,Age_y,0.000000
5,Hometown_y,0.000000
6,RPlace,0.000000
7,Season1,0.000000


In [25]:
z[z["Occupation"].isna()]

,First_Name_x,First_Name_y,Age_x,Hometown_x,Age_y,Hometown_y,RPlace,Season1,Relation,Occupation,profile_txt,profile_txt2
2,Pat,Brenda,43,"Landenberg, Pennsylvania",42,"Elkton, Maryland",9,1,Working moms,NaN,"[""I'd most like to receive a letter from: My k...",['no profile info']
7,Frank,Margarita,30,"Queens, New York",28,"Queens, New York",2,1,Separated couple,NaN,"[""I'd most like to receive a letter from: My d...",['no profile info']


In [26]:
z[z["Relation"].isna()]


,First_Name_x,First_Name_y,Age_x,Hometown_x,Age_y,Hometown_y,RPlace,Season1,Relation,Occupation,profile_txt,profile_txt2
245,Brodie,Kurt,28,"Dallas, Texas",30,"Dallas, Texas",5,28,NaN,Social Media Creator (Brodie)<br/>National Acc...,"[""Name: Kurt Gibson\nAge: 30\nTwitter: @kurteg...",['no profile info']


# Cleaning Scraped Data/Building a Dataset

### To Do's
* Clean up/merge columns with same meaning: example if LGBT and lgbt, need to be same column
* Fill na's 
* Clean text/tokenize Relation and Occupation fields
    * for missing, first see if these can be filled using profile text
* Merge profile columns where necessary. Clean text/tokenize the field. This is lower priority as it may be more advance analysis
* strings to integers where necessary


In [28]:
# Go through and group related columns to streamline features
'Co-ed teams'
'LGBT teams', 'Lesbian teams', 'Gay Teams', 'Gay/Bisexual Teams', 'Lesbian teams]','LGBT teams]', 'Gay Teams]'
'African-American Teams', 'African-American teams' # don't hold same info
'Straight/Gay Teams', 'Straight/Bisexual Teams', 'Straight/Bisexual Teams]'
'Friend teams', 'Work-related teams', 'Work-related teams]', 'Friend teams]',
'Cousins teams', 'Sibling teams', 'Parent/Child teams', 'Twin sibling teams'
'Stranger teams', 'Blind Dating teams', 'Stranger teams]'
'All-female teams'
'All-male teams'
'Asian-American Teams'
'Engaged teams', 'Dating teams','Married teams', 'Exes teams', 'Dating teams]'
'Interracial Teams', 'Interracial Teams]'
'Disabled teams]'
'Reality Show Teams', 'Reality Show Teams]'
'Composite teams'
'Returning racers'
'Hispanic-American Teams'

'Hispanic-American Teams'

In [29]:
# get relevant columns that don't require grouping/not team characteristics
dff = dat1[['Season1','RPlace','Name_x', 'Name_y','Hometown_x','Hometown_y','Age_x', 'Age_y', 'Relation', 'Occupation', 'profile_txt', 'profile_txt2']].copy()
dff.rename(columns={'Season1':'Season', 'Rplace':'Placement'}, inplace=True)
dff.head()
#.sort_values(['Name_x', 'Season','Placement'])

,Season,RPlace,Name_x,Name_y,Hometown_x,Hometown_y,Age_x,Age_y,Relation,Occupation,profile_txt,profile_txt2
0,1,11,Matt Robar,Ana Robar,"Simsbury, Connecticut","Simsbury, Connecticut",28,28,Married,Military folk,"[""I'd most like to receive a letter from: Nobo...",['no profile info']
1,1,10,Kim Smith,Leslie Kellner,"Baytown, Texas","Baytown, Texas",28,27,Roommates,Teachers,"[""I'd most like to receive a letter from: Caro...",['no profile info']
2,1,9,Pat Pierce,Brenda Mehta,"Landenberg, Pennsylvania","Elkton, Maryland",43,42,Working moms,NaN,"[""I'd most like to receive a letter from: My k...",['no profile info']
3,1,8,Dave Groark,Margaretta Groark,"Rockwall, Texas","Rockwall, Texas",65,60,Married,Air Force Pilot (Dave)<p>Retired (Margaretta)</p>,"[""I'd most like to receive a letter from: My k...",['no profile info']
4,1,7,Paul Alessi,Amie Barsky,"Los Angeles, California","Los Angeles, California",32,27,Engaged/cohabiting couple,Actor (Paul)<br/>Bartender (Amie),"[""I'd most like to receive a letter from: Brot...",['no profile info']


In [30]:
len(dff)

313

### Let's look at cleaning up occupation

1) need to remove the person's name... it will be inside parenthesis 
2) need to split the column where relevant to get jobs for each teammate. use '<br/>'
3) need a system to encode these to something meaningful...

In [31]:
dff['Occupation'] = dff['Occupation'].fillna('Missing')

# remove people's name in parenthesis
dff["Occupation"] = dff["Occupation"].apply(lambda x: re.sub(r'\(.*?\)', '', x))

In [32]:
# new data frame with split value columns
new = dff["Occupation"].str.split("<br/>", n=1, expand=True)

# do it again
new2 = new[0].str.split("<p>", n=1, expand=True)

dff["Occupation_x"] = new2[0]
dff["Occupation_y"] = np.where(new[1].notnull(), 
                               new[1],
                               new2[1])


dff["Occupation_y"] = np.where( (dff["Occupation_x"] != 'Missing') & (dff["Occupation_y"].isnull()), 
                                dff["Occupation_x"], dff["Occupation_y"])
    
dff["Occupation_y"] = dff["Occupation_y"].fillna('Missing')
dff["Occupation_y"] = dff["Occupation_y"].apply(lambda x: re.sub(r'\<.*?\>', '', x))

In [33]:
dff.tail()

,Season,RPlace,Name_x,Name_y,Hometown_x,Hometown_y,Age_x,Age_y,Relation,Occupation,profile_txt,profile_txt2,Occupation_x,Occupation_y
308,35,5,Ashlie Martin,Todd Martin,"Chino, California","Chino, California",38,38,Married,Special Education Teacher <br/>Hospitality Acc...,[],['no profile info'],Special Education Teacher,Hospitality Account Manager
309,35,4,Steve Cargile,Anna Leigh Wilson,"Petty, Texas","Royse City, Texas",54,28,Father/Daughter,Paint Contractor/ Speech Pathologist <br/>Priv...,['no profile info'],['no profile info'],Paint Contractor/ Speech Pathologist,Private Flight Attendant/Influencer
310,35,3,Rob McArthur,Corey McArthur,"Riverside, California","New York, New York",48,25,Father/Son,Teaching Assistant/Assistant Football Coach <b...,['Notes:\n\n1: Leg 7 featured two Roadblocks. ...,['no profile info'],Teaching Assistant/Assistant Football Coach,Senior Manager of client strategy analytics
311,35,2,Joel Strasser,Garrett Smith,"Kuna, Idaho","Meridian, Idaho",42,43,Best Friends,Auto Claims Training Specialist <br/>Delivery ...,['Notes:\n\n1: Leg 7 featured two Roadblocks. ...,['no profile info'],Auto Claims Training Specialist,Delivery Driver
312,35,1,Greg Franklin,John Franklin,"New York, New York","Mountain View, California",25,27,Brothers,Software Developer <br/>Product Manager,['no profile info'],['no profile info'],Software Developer,Product Manager


### Combine the Categorical Team Characteristics data Into Dataset

* combine relevant groups into one var
* get rid of NAs

In [42]:
# then can add in our team characteristic vars, which is 1 if relevant
# fill na's with 0


dff['LGBT_team'] = np.where(~dat1['LGBT teams'].isna(),1,
                             np.where(~dat1['Lesbian teams'].isna(), 1,
                                     np.where(~dat1['Gay Teams'].isna(), 1,
                                             np.where(~dat1['Gay/Bisexual Teams'].isna(), 1,
                                                     np.where(~dat1['Lesbian teams]'].isna(), 1,
                                                             np.where(~dat1['Gay/Bisexual Teams'].isna(), 1,
                                                                     np.where(~dat1['LGBT teams]'].isna(), 1,
                                                                             0) ))))))  

dff['AficanAm_team'] = np.where(~dat1['African-American Teams'].isna(), 1, 
                                np.where(~dat1['African-American teams'].isna(), 1,
                                        0))

dff['Friend_team'] = np.where(~dat1['Friend teams'].isna(), 1, 
                               np.where(~dat1['Work-related teams'].isna(), 1,
                                        np.where(~dat1['Work-related teams]'].isna(), 1, 
                                                 np.where(~dat1['Friend teams]'].isna(), 1,
                                                          0))))

dff['Family_team'] = np.where(~dat1['Parent/Child teams'].isna(), 1, 
                               np.where(~dat1['Twin sibling teams'].isna(), 1,
                                        np.where(~dat1['Sibling teams'].isna(), 1,
                                                 np.where(~dat1['Cousins teams'].isna(), 1,
                                                         0))))

dff['Stranger_team'] = np.where(~dat1['Stranger teams'].isna(), 1,
                                 np.where(~dat1['Stranger teams]'].isna(), 1,
                                          np.where(~dat1['Stranger teams]'].isna(), 1,
                                                   np.where(~dat1['Blind Dating teams'].isna(), 1,
                                                           0))))
dff['Female_team'] = np.where(~dat1['All-female teams'].isna(), 1, 0)
dff['Male_team'] = np.where(~dat1['All-male teams'].isna(), 1, 0)
dff['Coed_team'] = np.where(~dat1['Co-ed teams'].isna(), 1, 0)
dff['AsianAm_team'] = np.where(~dat1['Asian-American Teams'].isna(), 1, 0)
dff['Married_team'] = np.where(~dat1['Married teams'].isna(), 1, 0)
dff['Disabled_team'] = np.where(~dat1['Disabled teams]'].isna(), 1, 0)
dff['Hispanic_team'] = np.where(~dat1['Hispanic-American Teams'].isna(), 1, 0)
                                       
dff['Couple_or_Ex_team'] = np.where(~dat1['Engaged teams'].isna(), 1,
                                     np.where(~dat1['Dating teams'].isna(), 1,
                                              np.where(~dat1['Dating teams]'].isna(), 1,
                                                       np.where(~dat1['Exes teams'].isna(), 1,
                                                               0))))

dff['Interracial_team'] = np.where(~dat1['Interracial Teams'].isna(), 1,
                                    np.where(~dat1['Interracial Teams]'].isna(), 1,
                                            0))

dff['RealityShow_team'] = np.where(~dat1['Reality Show Teams'].isna(), 1,
                                    np.where(~dat1['Reality Show Teams]'].isna(), 1, 
                                            0))



In [43]:
# Target variable will be top 3 for now
dff['Top_3'] = np.where(dff['RPlace'] <= 3, 1, 0)
dff['Y'] = np.where(dff['RPlace'] == 1, 1,0)

In [44]:
dff.head()

,Season,RPlace,Name_x,Name_y,Hometown_x,Hometown_y,Age_x,Age_y,Relation,Occupation,profile_txt,profile_txt2,Occupation_x,Occupation_y,v1,v2,idx_occ_v1,idx_occ_v2,ohe_occ_v1,ohe_occ_v2,LGBT_team,AficanAm_team,Friend_team,Family_team,Stranger_team,Female_team,Male_team,Coed_team,AsianAm_team,Married_team,Disabled_team,Hispanic_team,Couple_or_Ex_team,Interracial_team,RealityShow_team,Top_3,Y
0,1,11,Matt Robar,Ana Robar,"Simsbury, Connecticut","Simsbury, Connecticut",28,28,Married,Military folk,"[""I'd most like to receive a letter from: Nobo...",['no profile info'],Military folk,Military folk,"[militari, folk]","[militari, folk]","[286.0, 181.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[286.0, 181.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,1,10,Kim Smith,Leslie Kellner,"Baytown, Texas","Baytown, Texas",28,27,Roommates,Teachers,"[""I'd most like to receive a letter from: Caro...",['no profile info'],Teachers,Teachers,[teacher],[teacher],"[457.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[457.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,9,Pat Pierce,Brenda Mehta,"Landenberg, Pennsylvania","Elkton, Maryland",43,42,Working moms,Missing,"[""I'd most like to receive a letter from: My k...",['no profile info'],Missing,Missing,[miss],[miss],"[288.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[288.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,8,Dave Groark,Margaretta Groark,"Rockwall, Texas","Rockwall, Texas",65,60,Married,Air Force Pilot <p>Retired </p>,"[""I'd most like to receive a letter from: My k...",['no profile info'],Air Force Pilot,Retired,"[air, forc, pilot]",[retir],"[17.0, 183.0, 342.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[392.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0
4,1,7,Paul Alessi,Amie Barsky,"Los Angeles, California","Los Angeles, California",32,27,Engaged/cohabiting couple,Actor <br/>Bartender,"[""I'd most like to receive a letter from: Brot...",['no profile info'],Actor,Bartender,[actor],[bartend],"[2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[48.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0


# Create and Save Dataset

In [45]:
dff['Avg_Age'] = np.mean([dff['Age_x'], dff['Age_y']],axis=0)
dff['Age_Diff'] = abs(dff['Age_x'] - dff['Age_y'])

In [46]:
# Do winners come from a particular state?
# is it better if both people are from the same state?
dff['State_x'] = dff.Hometown_x.apply(lambda x: x.split(",")[1])
dff['State_y'] = dff.Hometown_y.apply(lambda x: x.split(",")[1])
dff['Same_State'] = dff['State_x'] == dff['State_y']

In [47]:
features = ['Age_x', 'Age_y', 'LGBT_team',
       'AficanAm_team', 'Friend_team', 'Family_team', 'Stranger_team',
       'Female_team', 'Male_team', 'Coed_team', 'AsianAm_team', 'Married_team',
       'Disabled_team', 'Hispanic_team', 'Couple_or_Ex_team',
       'Interracial_team', 'RealityShow_team']

In [49]:
# export dataset
f = features + ['Season', 'RPlace', 'Name_x', 'Name_y', 'Hometown_x', 'Hometown_y', 
                'Avg_Age', 'Age_Diff','State_x','State_y','Same_State',
                'Occupation_x', 'Occupation_y', 'profile_txt', 'profile_txt2', 'Top_3', 'Y']

dff[f].to_csv("AmzingRceTeams_Data.csv", index=False)